In [ ]:
%load_ext autoreload

# 引入必要的库

In [ ]:
%autoreload
import sys,os
import numpy as np
import matplotlib.pyplot as plt
from tianmoucv.isp import fourdirection2xy
from tianmoucv.proc.reconstruct import laplacian_blending
import torch

from tianmoucv.data import TianmoucDataReader
import torch.nn.functional as F
import cv2

train='/data/lyh/tianmoucData/tianmoucReconDataset/train/'
dirlist = os.listdir(train)
traindata = [train + e for e in dirlist]

val='/data/lyh/tianmoucData/tianmoucReconDataset/test/'
vallist = os.listdir(val)
valdata = [val + e for e in vallist]
key_list = []

print('---------------------------------------------------')
for sampleset in traindata:
    print('---->',sampleset,'有：',len(os.listdir(sampleset)),'个样本')
    for e in os.listdir(sampleset):
        print(e)
        key_list.append(e)
print('---------------------------------------------------')
for sampleset in valdata:
    print('---->',sampleset,'有：',len(os.listdir(sampleset)),'个样本')
    for e in os.listdir(sampleset):
        print(e)
        key_list.append(e)
        
all_data = valdata + traindata


In [ ]:
key_list = ['underbridge_hdr_4']

# 融合图像

In [ ]:
%autoreload
import torch.nn as nn
import math
import time

MSE_LossFn = nn.MSELoss()
speedUpRate = 1

def images_to_video(frame_list,name,size=(640,320),Flip=False):
    fps = 25        
    ftmax = max([np.max(ft) for ft in frame_list])
    ftmin = min([np.min(ft) for ft in frame_list])
    out = cv2.VideoWriter(name,0x7634706d , fps, size)
    for ft in frame_list:
        ft = (ft-ftmin)/(ftmax-ftmin)
        ft2 = (ft*255).astype(np.uint8)
        out.write(ft2)
    out.release()

psnrcount =0
count = 0

for key in key_list:
    dataset = TianmoucDataReader(all_data,MAXLEN=500*speedUpRate,matchkey=key,speedUpRate=speedUpRate)
    dataLoader = torch.utils.data.DataLoader(dataset, batch_size=1,\
                                          num_workers=4, pin_memory=False, drop_last = False)

    PSNR = 0
    img_list = []
    for index,sample in enumerate(dataLoader,0):
        if index<0:
            continue
        if index<= 1:
            psnrcount += 1
            F0 = sample['F0'][0,...]
            F0_HDR = sample['F0_HDR'][0,...]

            raw_F0 = F0.clone()

            
            for t in [0]:
                tsdiff = sample['rawDiff'][0,...]/128.0        
                SD = tsdiff[1:,t,...].permute(1,2,0)
                ts= time.time()
                Ix,Iy= fourdirection2xy(SD)
                Ix = F.interpolate(torch.Tensor(Ix).unsqueeze(0).unsqueeze(0), size=(320,640), mode='bilinear').squeeze(0).squeeze(0)
                Iy = F.interpolate(torch.Tensor(Iy).unsqueeze(0).unsqueeze(0), size=(320,640), mode='bilinear').squeeze(0).squeeze(0)
                blend_hdr = laplacian_blending(-Ix,-Iy, srcimg= F0,iteration=20, mask_rgb=True,mask_th=36)
                blend_hdr_more_sd = laplacian_blending(-Ix,-Iy, srcimg= F0,iteration=20, mask_rgb=True,mask_th=48)#更激进的参数
                te = time.time()
                print('cost:',te-ts,'s')
                
                show = torch.cat([raw_F0,blend_hdr],dim=0)
            
                img_list.append(show.numpy()[...,[2,1,0]])
            
            
            plt.figure(figsize=(12,6))
            plt.subplot(1,3,1)  
            plt.imshow(F0_HDR,cmap='gray')
            plt.subplot(1,3,2)  
            plt.imshow(blend_hdr)
            plt.subplot(1,3,3)  
            plt.imshow(blend_hdr_more_sd)
            plt.show()
        else:
            break
    
    #images_to_video(img_list,size=(640,640),name='./Direct_'+key+'.mp4')